# Recommender system for a medical supplies company

The goal of the project is to help the company generate incremental sales by
implementing a recommender system.
Data Prep and Modelling
+ Missing data
+ Calculate find the most popular product using maximum sales by volume and dollar amount
+ Company with a maximum purchase
+ Build a popularity-based recommender system
+ Build a recommender system using matrix factorization
+ Cosine similarity for product description

In [1]:
#importing required packlages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF

In [2]:
#importing data
df = pd.read_csv('PBL 5 recommendation data.csv', encoding='latin1')

/var/folders/7w/sfqhq6jd47gg5c2xyhv9229w0000gn/T/ipykernel_2469/2258884698.py:2: DtypeWarning: Columns (20,33,73,106,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('PBL 5 recommendation data.csv', encoding='latin1')


# EDA - Explorary Data Analysis

In [3]:
df.head()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,...,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF61071,0.0,NaN
1,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF02132,NaN,NaN
2,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2 x Extra large,Nitrile,NaN,NaN,NaN,PF00342,0.0,NaN
3,4,James,Anderson,NaN,1386780263,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF04970,NaN,NaN
4,5,Abraham,Pollak,Company3,1386861599,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF03045,NaN,NaN


In [4]:
df.shape

(4194, 181)

In [5]:
df.isnull().sum()

Customers.id                0
Customers.fname             0
Customers.lname             0
Customers.company        3467
Customers.create_date       0
                         ... 
Products.leg_style       4194
Products.seat_size       4032
Products.family_id        325
Products.saved_status     264
Products.freight_cost    4194
Length: 181, dtype: int64

In [6]:
df.describe()

,Customers.id,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,...,Products.hygienic,Products.default_quantity,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.google_shopping_label,Products.leg_style,Products.saved_status,Products.freight_cost
count,4194.000000,4.194000e+03,101.0,2114.000000,0.0,5.000000,3.000000,0.0,0.0,0.0,...,2080.000000,0.0,3998.000000,3998.000000,3998.000000,659.000000,387.000000,0.0,3930.000000,0.0
mean,1796.518121,1.440971e+09,0.0,0.979659,NaN,0.200000,7.333333,NaN,NaN,NaN,...,0.064423,NaN,17.208216,10.872750,11.592447,15.487041,3.863049,NaN,0.876336,NaN
std,1065.679090,1.887681e+07,0.0,0.141196,NaN,0.447214,0.577350,NaN,NaN,NaN,...,0.245564,NaN,10.508782,6.781847,10.250909,22.256617,2.098446,NaN,1.364374,NaN
min,3.000000,1.386089e+09,0.0,0.000000,NaN,0.000000,7.000000,NaN,NaN,NaN,...,0.000000,NaN,0.000000,0.000000,0.000000,0.020000,0.000000,NaN,0.000000,NaN
25%,865.250000,1.427323e+09,0.0,1.000000,NaN,0.000000,7.000000,NaN,NaN,NaN,...,0.000000,NaN,9.000000,5.700000,4.000000,2.775000,5.000000,NaN,0.000000,NaN
50%,1747.500000,1.446412e+09,0.0,1.000000,NaN,0.000000,7.000000,NaN,NaN,NaN,...,0.000000,NaN,15.500000,10.000000,8.645000,8.900000,5.000000,NaN,0.000000,NaN
75%,2698.750000,1.456955e+09,0.0,1.000000,NaN,0.000000,7.500000,NaN,NaN,NaN,...,0.000000,NaN,23.500000,15.000000,15.000000,21.000000,5.000000,NaN,3.000000,NaN
max,3736.000000,1.463418e+09,0.0,1.000000,NaN,1.000000,8.000000,NaN,NaN,NaN,...,1.000000,NaN,90.000000,80.000000,58.000000,175.000000,5.000000,NaN,3.000000,NaN


In [7]:
df.sample(10)

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,...,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
350,312,Anne,Montez,NaN,1409424107,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,Polypropylene Backing,NaN,NaN,NaN,PF01462,0.0,NaN
2184,1817,Erica,Smith,NaN,1448294336,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Chrome Plated Steel,NaN,NaN,NaN,PF01127,0.0,NaN
921,767,Anabel,Martinez,NaN,1425202713,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,Youth,NaN,NaN,NaN,NaN,CR003,0.0,NaN
827,684,ramiro,torres,NaN,1423166302,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Desk Length Removable - Swing Away Foot Rest,NaN,"20"" x 16""",PF05428,3.0,NaN
145,147,Kay,Pruitk,NaN,1400164746,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,PF21897,3.0,NaN
1838,1533,Monty,Ferbert,Company139,1441329647,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,Large/Extra Large,NaN,NaN,NaN,NaN,PF01409,0.0,NaN
2276,1914,Donald,Golighely,NaN,1449692314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Aluminum,NaN,NaN,NaN,PF04774,3.0,NaN
3608,3152,CLARENCE,POLLARD,Company356,1460244938,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF08485,3.0,NaN
331,304,Alexander,Manly,NaN,1409081056,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3151,2704,susan,creek,NaN,1457024257,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,118.00 ML,NaN,NaN,NaN,NaN,PF06506,3.0,NaN


In [8]:
#Creating 4 separate dataframes from the main DataFrame by 'Customers' , 'Orders', 'Order Items' and'Products'
customers_df = df.filter(regex='^Customers\.')
orders_df = df.filter(regex='^Orders\.')
products_df = df.filter(regex='^Products\.')
order_items_df = df.filter(regex='^Order_Items\.')   

customers_df.columns = customers_df.columns.str.replace('Customers\.', '', regex=True)
orders_df.columns = orders_df.columns.str.replace('Orders\.', '', regex=True)
products_df.columns = products_df.columns.str.replace('Products\.', '', regex=True)
order_items_df.columns = order_items_df.columns.str.replace('Order_Items\.', '', regex=True)
#gitHub:zafermesutbilen

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
/var/folders/7w/sfqhq6jd47gg5c2xyhv9229w0000gn/T/ipykernel_2469/1874787618.py:2: SyntaxWarning: invalid escape sequence '\.'
  customers_df = df.filter(regex='^Customers\.')
/var/folders/7w/sfqhq6jd

In [9]:
customers_df = customers_df[['id', 'fname', 'lname', 'company']]

In [10]:
customers_df.head()

,id,fname,lname,company
0,797,Christy,Dill,Company0
1,3,John,Smith,Company1
2,3,John,Smith,Company1
3,4,James,Anderson,NaN
4,5,Abraham,Pollak,Company3


In [11]:
orders_df.head()

,id,customer_id,fname,lname,company,order_number,reorder_id,external_source,external_id,currency,...,shipped_date,comments,notes,registry_id,gift_message,website,mailing,flags,partial_ship,customer_type
0,3758,797,Christy,Dill,Company0,3758,NaN,NaN,NaN,USD,...,1.426101e+09,NaN,Insured By Eye4Fraud,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,23,3,John,Smith,Company1,23,NaN,NaN,NaN,USD,...,1.386103e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,9531,3,John,Smith,Company1,9531,NaN,NaN,NaN,USD,...,NaN,test order,Test,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,29,4,James,Anderson,NaN,29,NaN,Ebay,NaN,USD,...,1.386800e+09,ebay order\rpaid by pay pal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,30,5,Abraham,Pollak,Company3,30,NaN,NaN,NaN,USD,...,1.386876e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [12]:
orders_df = orders_df[['id', 'customer_id', 'currency', 'subtotal']]

In [13]:
orders_df.head()

,id,customer_id,currency,subtotal
0,3758,797,USD,57.20
1,23,3,USD,20.00
2,9531,3,USD,68.78
3,29,4,USD,19.56
4,30,5,USD,36.05


In [14]:
products_df.head()

,id,status,product_type,template,vendor,import_id,name,display_name,menu_name,list_price,...,google_shopping_label,product_option,size,material,arm_style,leg_style,seat_size,family_id,saved_status,freight_cost
0,2310.0,0.0,NaN,sub-product-detail.html,1.0,3.0,"Basic Steel Rollators,Green","Basic Lightweight Rollators For Adults, With S...",NaN,80.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF61071,0.0,NaN
1,177.0,1.0,NaN,sub-product-detail.html,1.0,3.0,Urinary Drain Bags,Urinary Drain Bags,NaN,14.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF02132,NaN,NaN
2,1.0,0.0,NaN,sub-product-detail.html,1.0,3.0,"SensiCare Nitrile Exam Gloves,Blue,XX-Large","SensiCare Nitrile Exam Gloves, Blue, XX-Large",NaN,96.29,...,NaN,NaN,2 x Extra large,Nitrile,NaN,NaN,NaN,PF00342,0.0,NaN
3,983.0,0.0,NaN,sub-product-detail.html,1.0,3.0,Basket for 2-Button Walkers,Basket for 2-Button Walkers,NaN,27.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF04970,NaN,NaN
4,991.0,1.0,NaN,sub-product-detail.html,1.0,3.0,TENS 3000 Analog Unit,TENS 3000 Analog Unit,NaN,50.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF03045,NaN,NaN


In [15]:
products_df = products_df[['id','name','sale_price', 'shopping_brand']]

In [16]:
customers_df.head()

,id,fname,lname,company
0,797,Christy,Dill,Company0
1,3,John,Smith,Company1
2,3,John,Smith,Company1
3,4,James,Anderson,NaN
4,5,Abraham,Pollak,Company3


In [17]:
customers_df.shape

(4194, 4)

In [18]:
customers_df.drop_duplicates(inplace=True)

In [23]:
products_df.drop_duplicates(inplace=True)

In [25]:
#Sales Dataframe
sales_df = df[['Orders.id', 'Customers.id', 'Customers.company', 'Orders.currency', 'Orders.subtotal', 'Order_Items.product_id', 'Order_Items.qty', 'Order_Items.price','Products.name', 'Products.shopping_brand']]

In [26]:
sales_df.head()

,Orders.id,Customers.id,Customers.company,Orders.currency,Orders.subtotal,Order_Items.product_id,Order_Items.qty,Order_Items.price,Products.name,Products.shopping_brand
0,3758,797,Company0,USD,57.20,2310.0,1,57.20,"Basic Steel Rollators,Green",MEDLINE
1,23,3,Company1,USD,20.00,177.0,4,5.00,Urinary Drain Bags,MEDLINE
2,9531,3,Company1,USD,68.78,1.0,1,68.78,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",MEDLINE
3,29,4,NaN,USD,19.56,983.0,1,19.56,Basket for 2-Button Walkers,MEDLINE
4,30,5,Company3,USD,36.05,991.0,1,36.05,TENS 3000 Analog Unit,ROSCOE MEDICAL INC


In [27]:
sales_df = sales_df[sales_df['Order_Items.product_id'].notnull()]

sales_df['Order_Items.product_id'] = sales_df['Order_Items.product_id'].astype(int)

# Popularitry Based Recommender

In [30]:
populer_df = df.groupby('Products.name').agg({
    'Order_Items.qty': 'sum',
    'Order_Items.price': 'sum'
}).reset_index()
populer_product_df = populer_df.sort_values(by='Order_Items.qty', ascending=False)

In [31]:
populer_product_df.head()

,Products.name,Order_Items.qty,Order_Items.price
398,"Disposable Emergency Blanket,Not Applicable",396,622.98
803,"MoliCare Disposable Super Plus Briefs,27""-47""",228,3907.16
9,"2-Ply Tissue/Poly Professional Towels,Not Appl...",220,50.54
1048,Sanitary Pads with Adhesive & Wings,133,84.64
477,"Emesis Bags,Blue,36.000 OZ",129,2283.47


# Companies With the Maximum Purchases

In [33]:
max_seller = df.groupby('Customers.company').agg({'Order_Items.qty': 'sum'}).reset_index()
max_seller_df = max_seller.sort_values(by='Order_Items.qty', ascending=False)

In [34]:
max_seller_df.head()

,Customers.company,Order_Items.qty
380,Company59,381
271,Company343,133
410,Company86,105
36,Company130,71
81,Company171,47


In [35]:
max_seller_df.sample(10)

,Customers.company,Order_Items.qty
106,Company194,2
307,Company376,10
153,Company237,1
276,Company348,2
280,Company351,1
242,Company317,4
136,Company221,1
9,Company106,1
252,Company326,1
200,Company28,3


# Popularity Based Recommendation System

In [37]:
user_item_matrix = df.pivot_table(index='Customers.id', 
                                   columns='Products.name', 
                                   values='Order_Items.qty', 
                                   fill_value=0)
#this recommender systemn is from POE

# Matrix Based Recommendation System

In [38]:
model = NMF(n_components=10, random_state=42)
user_features = model.fit_transform(user_item_matrix)
item_features = model.components_

In [39]:
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, 
                                   index=user_item_matrix.columns, 
                                   columns=user_item_matrix.columns)

In [40]:
def get_recommendations(product_name, n_recommendations=5):
    if product_name not in item_similarity_df.columns:
        return f"{product_name} product not found."
    

In [44]:
product_name = 'Emesis Bag' 
recommendations = get_recommendations(product_name)
print(f"{product_name} için öneriler:\n{recommendations}")

Emesis Bag için öneriler:
Emesis Bag product not found.


In [45]:
product_name = 'Disposable Emergency Blanket,Not Applicable' 
recommendations = get_recommendations(product_name)
print(f"{product_name} için öneriler:\n{recommendations}")

Disposable Emergency Blanket,Not Applicable için öneriler:
None
